forked from https://www.kaggle.com/code/siddhvr/foldseek-blastp-ensemble

just to be noted:

this is the same notebook which was submitted in CAFA5 for demonstration purpose (not much changes have been made)
it utilizes Foldseek/Blasp model finetuned on CAFA5 dataset (try training the model on CAFA6)
submission utilizes ensemble for cafa5 notebook (scroll all the way down to check)

Since, running Foldseek on Kaggle gives MMseq2, I directly used the foldseek test set submission. You can refer to this amazing notebook on Foldseek by :RAMAN for the entire code https://www.kaggle.com/code/samusram/leveraging-foldseek

In [1]:
# !conda install -c bioconda blast -y

In [2]:
# !wget https://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ncbi-blast-2.17.0+-x64-linux.tar.gz

In [3]:
# !tar zxvpf ncbi-blast-2.17.0+-x64-linux.tar.gz

In [4]:
# !cp /kaggle/working/ncbi-blast-2.17.0+/bin/* /opt/conda/bin

In [5]:
# !pip install git+https://github.com/SamusRam/ProFun.git

In [6]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm, trange
from Bio import SeqIO
import numpy as np

from profun.models import BlastMatching, BlastConfig
from profun.utils.project_info import ExperimentInfo

## Obtaining train data

In [22]:
data_root = Path('data/raw/cafa6')
out_root = Path('output/blast')
train_terms = pd.read_csv(data_root/"Train/train_terms.tsv",sep="\t")

ids = []
seqs = []
with open(data_root/"Train/train_sequences.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        ids.append(record.id.split('|')[1])
        seqs.append(str(record.seq))
train_seqs_df = pd.DataFrame({'EntryID': ids, 'Seq': seqs})
train_df_long = train_terms.merge(train_seqs_df, on='EntryID')

## Init model

In [10]:
import os
import shutil

# Kiểm tra xem hệ thống có tìm thấy lệnh này không
blast_path = shutil.which("makeblastdb")
print(f"Hệ thống tìm thấy makeblastdb tại: {blast_path}")

# Nếu là None, ta cần tìm thủ công trong môi trường Conda của bạn
if blast_path is None:
    # Dựa vào log lỗi của bạn, đường dẫn environment là: ~/.conda/envs/cafa
    # Nên file thực thi thường nằm ở thư mục /bin bên trong đó
    potential_path = os.path.expanduser("~/.conda/envs/cafa/bin/makeblastdb")
    if os.path.exists(potential_path):
        print(f"Tìm thấy thủ công tại: {potential_path}")
    else:
        print("Vẫn không tìm thấy file. Có lẽ cài đặt bị lỗi.")

Hệ thống tìm thấy makeblastdb tại: None
Tìm thấy thủ công tại: /data/hien/.conda/envs/cafa/bin/makeblastdb


In [11]:
import os

# Lấy đường dẫn đến môi trường Conda hiện tại (từ log lỗi của bạn)
conda_env_path = os.path.expanduser("~/.conda/envs/cafa")
conda_bin_path = os.path.join(conda_env_path, "bin")

# Thêm thư mục bin của conda vào biến môi trường PATH của Python hiện tại
if conda_bin_path not in os.environ['PATH']:
    os.environ['PATH'] = conda_bin_path + os.pathsep + os.environ['PATH']
    print(f"Đã thêm {conda_bin_path} vào PATH.")

# Kiểm tra lại lần cuối
import shutil
if shutil.which("makeblastdb"):
    print("OK! Đã nhận diện được lệnh makeblastdb via subprocess.")
else:
    print("Cảnh báo: Vẫn chưa nhận diện được lệnh.")

Đã thêm /data/hien/.conda/envs/cafa/bin vào PATH.
OK! Đã nhận diện được lệnh makeblastdb via subprocess.


In [35]:
experiment_info = ExperimentInfo(validation_schema='public_lb', 
                                 model_type='blast', model_version='1nn')

config = BlastConfig(experiment_info=experiment_info, 
                     id_col_name='EntryID', 
                     target_col_name='term', 
                     seq_col_name='Seq', 
                     class_names=list(train_df_long['term'].unique()), 
                     optimize_hyperparams=False, 
                     n_calls_hyperparams_opt=None,
                    hyperparam_dimensions=None,
                    per_class_optimization=None,
                    class_weights=None,
                    n_neighbours=5,
                    e_threshold=0.02,
                     n_jobs=100,
                     pred_batch_size=10
                    )

blast_model = BlastMatching(config)

## Train model

In [36]:
blast_model.fit(train_df_long)

## Predict on test
It's an illustration. I computed the predictions for the whole test set offline. Unfortunately, on Kaggle I experience MMseq2 error reported [here](https://github.com/soedinglab/metaeuk/issues/48) (there's an OpenMP-related [check in MMSeq2](https://github.com/soedinglab/metaeuk/blob/1da320a9daa75dce5539442b5674f69951a2fe4f/lib/mmseqs/src/commons/CommandCaller.cpp#L17). If you experience the same error on your local machine, please refer to [this thread](https://github.com/soedinglab/metaeuk/issues/48)).

In [17]:
ids = []
seqs = []
with open(data_root/"Test/testsuperset.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        ids.append(record.id)
        seqs.append(str(record.seq))
test_seqs_df = pd.DataFrame({'EntryID': ids, 'Seq': seqs})
test_seqs_df

In [30]:
import warnings
import pandas as pd

# Tắt tất cả các warning chung
warnings.filterwarnings('ignore')

# Tắt riêng cái SettingWithCopyWarning của Pandas (cái bảng hồng trong ảnh)
pd.options.mode.chained_assignment = None

In [31]:
import logging

# Chỉ hiện lỗi nghiêm trọng, ẩn các thông báo INFO đi
logging.getLogger("profun").setLevel(logging.WARNING)

# Hoặc tắt log của toàn bộ hệ thống (nếu cần)
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
test_pred_df = blast_model.predict_proba(test_seqs_df.drop_duplicates('EntryID'), return_long_df=True)

Predicting with BLASTp-matching..:   0%|          | 0/22431 [00:00<?, ?it/s]

In [25]:
import os
res_path = out_root/"submission.tsv"
os.makedirs(out_root, exist_ok=True)

In [27]:
test_pred_df.to_csv(res_path)

In [ ]:
# submissions_merged = submission_best_public.merge(test_pred_df_blast, left_on=['Id', 'GO term'], 
#                                                   right_on=[1, 2], how='outer')

In [ ]:
# submissions_merged.drop([1, 2], axis=1, inplace=True)
# submissions_merged['confidence_combined'] = submissions_merged.apply(lambda row: row['Confidence'] if not np.isnan(row['Confidence']) else row[3], axis=1)


In [ ]:
# submissions_merged[['Id', 'GO term', 'confidence_combined']].to_csv('submission.tsv',
#     sep='\t', header=False, index=False)
